In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import Row
from pyspark import StorageLevel
from pyspark.sql import SparkSession
import sys
import numpy as np

In [2]:
# sc = SparkContext("local[*]", "NFL")
spark = SparkSession.builder.master("local[*]").appName("NFL").getOrCreate()

In [6]:
df = spark.read.option("header",True).csv('../in/PlayerTrackData.csv')
# rdd = sc.textFile('../in/PlayerTrackData.csv')

In [7]:
# show header
df.printSchema()
# rdd.take(1)

root
 |-- PlayKey: string (nullable = true)
 |-- time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: string (nullable = true)
 |-- s: string (nullable = true)



In [5]:
# remove header
# rdd = rdd.filter(lambda x: 'PlayKey' not in x)

In [6]:
# split the col
# rdd = rdd.map(lambda x: x.split(','))

In [7]:
# remove row if missing some value
# rdd = rdd.filter(lambda x: x[5] != '' and x[7] != '')

In [8]:
# convert to dataframe object
# sqlContext = SQLContext(sc)
# df = rdd.toDF(['PlayKey','time','event','x','y','dir','dis','o','s'])

# if df want to use rdd function
# just df.rdd.xxx()

In [8]:
df.select('PlayKey','s').show()

+---------+----+
|  PlayKey|   s|
+---------+----+
|26624-1-1|0.13|
|26624-1-1|0.12|
|26624-1-1|0.12|
|26624-1-1| 0.1|
|26624-1-1|0.09|
|26624-1-1|0.07|
|26624-1-1|0.05|
|26624-1-1|0.02|
|26624-1-1|0.01|
|26624-1-1|0.01|
|26624-1-1| 0.0|
|26624-1-1|0.01|
|26624-1-1|0.01|
|26624-1-1|0.01|
|26624-1-1| 0.0|
|26624-1-1|0.01|
|26624-1-1| 0.0|
|26624-1-1| 0.0|
|26624-1-1| 0.0|
|26624-1-1| 0.0|
+---------+----+
only showing top 20 rows



In [9]:
# get all ball_snap play record
ball_snap_tracks = df.where("event == 'ball_snap'")
# rename x,y,time for ball snap
drop_list = ['dir','dis','o','event','s']
ball_snap_tracks = ball_snap_tracks.withColumnRenamed('x','snap_x').withColumnRenamed('y','snap_y').withColumnRenamed('time','snap_time').drop(*drop_list)
# join two table (orign tracks and tracks only contained ball_snap event)
tracks = df.join(ball_snap_tracks, ['PlayKey'], "left_outer")
# remove any data for a play 0.1 second before snap
tracks = tracks.where('time >= (snap_time - 0.1)')
# tracks.persist(StorageLevel.MEMORY_ONLY)


In [11]:
cnt = tracks.count()
print(cnt)

37087421


In [12]:
# convert time to float
tracks = tracks.withColumn("time", tracks.time.cast('float').alias("time"))

In [13]:
last_event_tracks = tracks.where('event is not null').groupBy('PlayKey').agg(f.max('time').alias('time'))

In [15]:
last_event_tracks.show()
cnt = last_event_tracks.count()
print(cnt)

+-----------+----+
|    PlayKey|time|
+-----------+----+
|26624-13-32|23.6|
|26624-15-34|29.2|
|26624-17-30|20.1|
|26624-23-64|29.4|
|26624-27-32|21.8|
|26624-28-56|23.7|
| 26624-29-4|22.8|
|26624-29-80| 8.0|
|26624-30-11|18.2|
| 26624-6-58|40.3|
| 26624-8-75|18.1|
|27363-10-71|23.9|
|27363-15-21|17.2|
|27363-15-23|17.0|
|  27363-2-6|26.8|
|27363-22-39|14.5|
|27363-22-50|18.8|
|27363-29-60|30.1|
|27363-30-75| 7.9|
| 27363-6-23|24.5|
+-----------+----+
only showing top 20 rows

253227


In [16]:
# join tracks record and last event tracks record
last_event_tracks = last_event_tracks.join(tracks, ['PlayKey','time'], "inner")['PlayKey','time','event']

In [17]:
# rename col
last_event_tracks = last_event_tracks.withColumnRenamed('time','last_time').withColumnRenamed('event','last_event')

In [18]:
tracks = tracks.join(last_event_tracks, ['PlayKey'], 'left_outer')

In [19]:
# Remove any data for a play 0.1 second after last event
tracks = tracks.where('time <= (last_time + 0.1)')

In [20]:
tracks.persist(StorageLevel.MEMORY_ONLY)

DataFrame[PlayKey: string, time: float, event: string, x: string, y: string, dir: string, dis: string, o: string, s: string, snap_time: string, snap_x: string, snap_y: string, last_time: float, last_event: string]

In [21]:
tracks.show()
l = tracks.count()
print(l)

+-----------+----+---------+-----+-----+------+----+-----+----+---------+------+------+---------+--------------------+
|    PlayKey|time|    event|    x|    y|   dir| dis|    o|   s|snap_time|snap_x|snap_y|last_time|          last_event|
+-----------+----+---------+-----+-----+------+----+-----+----+---------+------+------+---------+--------------------+
|26624-13-32|19.4|     null|19.85| 23.9|267.19| 0.0|15.21|0.29|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.5|ball_snap|19.85| 23.9|266.62| 0.0|15.32|0.51|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.6|     null|19.84| 23.9|266.04| 0.0|15.48|0.68|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.7|     null|19.84| 23.9|265.88| 0.0|15.85| 0.8|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.8|     null|19.84| 23.9|265.98| 0.0|16.76|0.89|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.9|     null|19.83| 23.9|266.15|0

In [14]:
tracks.head(100)
tracks.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: float (nullable = true)
 |-- event: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: string (nullable = true)
 |-- s: string (nullable = true)
 |-- snap_time: string (nullable = true)
 |-- snap_x: string (nullable = true)
 |-- snap_y: string (nullable = true)
 |-- last_time: float (nullable = true)
 |-- last_event: string (nullable = true)



In [49]:
# only record the time from the start of first event-1 to the end of last event+1

In [57]:
# spark = SparkSession.builder.master("local[*]").appName("NFL").getOrCreate()
pl = spark.read.option("header",True).csv('../in/PlayList.csv')

In [58]:
# get season 2 distinct player key
playerkey = pl.select('PlayerKey').where('PlayerDay >= 350').distinct()
playerkey_list = playerkey.rdd.map(lambda x:x.PlayerKey).collect()

res = playerkey.collect()
cnt = playerkey.count()
print(res)
print(cnt)
print(ids)

[Row(PlayerKey='38252'), Row(PlayerKey='42370'), Row(PlayerKey='44423'), Row(PlayerKey='30953'), Row(PlayerKey='34347'), Row(PlayerKey='44860'), Row(PlayerKey='27363'), Row(PlayerKey='44067'), Row(PlayerKey='41084'), Row(PlayerKey='44629'), Row(PlayerKey='44825'), Row(PlayerKey='36579'), Row(PlayerKey='44934'), Row(PlayerKey='46066'), Row(PlayerKey='42359'), Row(PlayerKey='42615'), Row(PlayerKey='44548'), Row(PlayerKey='34214'), Row(PlayerKey='34259'), Row(PlayerKey='44900'), Row(PlayerKey='45945'), Row(PlayerKey='45953'), Row(PlayerKey='46074'), Row(PlayerKey='39680'), Row(PlayerKey='37068'), Row(PlayerKey='43483'), Row(PlayerKey='36672'), Row(PlayerKey='45987'), Row(PlayerKey='40405'), Row(PlayerKey='42399'), Row(PlayerKey='38325'), Row(PlayerKey='40286'), Row(PlayerKey='44421'), Row(PlayerKey='35617'), Row(PlayerKey='34243'), Row(PlayerKey='42588'), Row(PlayerKey='46316'), Row(PlayerKey='44482'), Row(PlayerKey='44037'), Row(PlayerKey='42346'), Row(PlayerKey='39771'), Row(PlayerKey='

In [59]:
# convert string to int
pl = pl.withColumn("PlayerDay", pl.PlayerDay.cast('int').alias("PlayerDay"))

In [60]:
# only select the player who play in season 2, dont know is he play season 1
pl = pl[pl.PlayerKey.isin(playerkey_list)]

In [61]:
# add season to df, if less than 350, season 1 else season 2
pl = pl.withColumn('season', f.when(f.col("PlayerDay") < 350,1).otherwise(2))

In [62]:
pl.show()
l = pl.count()
print(l)

+---------+-------+----------+--------------+---------+----------+-----------+---------+-----------+--------------+--------+--------------+--------+-------------+------+
|PlayerKey| GameID|   PlayKey|RosterPosition|PlayerDay|PlayerGame|StadiumType|FieldType|Temperature|       Weather|PlayType|PlayerGamePlay|Position|PositionGroup|season|
+---------+-------+----------+--------------+---------+----------+-----------+---------+-----------+--------------+--------+--------------+--------+-------------+------+
|    26624|26624-1| 26624-1-1|   Quarterback|        1|         1|    Outdoor|Synthetic|         63|Clear and warm|    Pass|             1|      QB|           QB|     1|
|    26624|26624-1| 26624-1-2|   Quarterback|        1|         1|    Outdoor|Synthetic|         63|Clear and warm|    Pass|             2|      QB|           QB|     1|
|    26624|26624-1| 26624-1-3|   Quarterback|        1|         1|    Outdoor|Synthetic|         63|Clear and warm|    Rush|             3|      QB|  

In [65]:
pl_map = pl[['PlayKey','GameID','season']]

In [66]:
pl_map.show()

+----------+-------+------+
|   PlayKey| GameID|season|
+----------+-------+------+
| 26624-1-1|26624-1|     1|
| 26624-1-2|26624-1|     1|
| 26624-1-3|26624-1|     1|
| 26624-1-4|26624-1|     1|
| 26624-1-5|26624-1|     1|
| 26624-1-6|26624-1|     1|
| 26624-1-7|26624-1|     1|
| 26624-1-8|26624-1|     1|
| 26624-1-9|26624-1|     1|
|26624-1-10|26624-1|     1|
|26624-1-11|26624-1|     1|
|26624-1-12|26624-1|     1|
|26624-1-13|26624-1|     1|
|26624-1-14|26624-1|     1|
|26624-1-15|26624-1|     1|
|26624-1-16|26624-1|     1|
|26624-1-17|26624-1|     1|
|26624-1-18|26624-1|     1|
|26624-1-19|26624-1|     1|
|26624-1-20|26624-1|     1|
+----------+-------+------+
only showing top 20 rows



In [68]:
map_tracks = tracks.join(pl_map,['PlayKey'],'left_outer')
map_tracks.persist(StorageLevel.MEMORY_ONLY)

DataFrame[PlayKey: string, time: float, event: string, x: string, y: string, dir: string, dis: string, o: string, s: string, snap_time: string, snap_x: string, snap_y: string, last_time: float, last_event: string, GameID: string, season: int]

In [69]:
map_tracks.where('PlayKey == "47888-13-55"').show()

+-----------+----+---------+-----+-----+------+----+------+----+---------+------+------+---------+----------+------+------+
|    PlayKey|time|    event|    x|    y|   dir| dis|     o|   s|snap_time|snap_x|snap_y|last_time|last_event|GameID|season|
+-----------+----+---------+-----+-----+------+----+------+----+---------+------+------+---------+----------+------+------+
|47888-13-55|12.0|     null|68.27|37.85| 52.23|0.02|263.13|0.04|     12.1| 68.27| 37.85|     19.0|    tackle|  null|  null|
|47888-13-55|12.1|ball_snap|68.27|37.85| 50.71| 0.0|263.13|0.03|     12.1| 68.27| 37.85|     19.0|    tackle|  null|  null|
|47888-13-55|12.2|     null|68.26|37.86| 33.34|0.02|263.13|0.03|     12.1| 68.27| 37.85|     19.0|    tackle|  null|  null|
|47888-13-55|12.3|     null|68.25|37.86| 24.49|0.01|265.05|0.02|     12.1| 68.27| 37.85|     19.0|    tackle|  null|  null|
|47888-13-55|12.4|     null|68.24|37.86|286.07|0.01|266.58|0.03|     12.1| 68.27| 37.85|     19.0|    tackle|  null|  null|
|47888-1

In [70]:
print(map_tracks.count())

17304958


In [73]:
# Fix orientation
# Assume other seasons based on direction at snap - if orientation at snap is outside normal range, shift.
s1 = map_tracks.select('PlayKey').where('event == "ball_snap" and o < 50 and season is null').distinct().collect()
s2 = map_tracks.select('PlayKey').where('event == "ball_snap" and o > 325 and season is null').distinct().collect()
s3 = map_tracks.select('PlayKey').where('event == "ball_snap" and o < 225 and o > 125 and season is null').distinct().collect()

In [72]:
map_tracks

DataFrame[PlayKey: string, time: float, event: string, x: string, y: string, dir: string, dis: string, o: string, s: string, snap_time: string, snap_x: string, snap_y: string, last_time: float, last_event: string, GameID: string, season: int]

In [75]:
def foo(x):
    return [i['PlayKey'] for i in x]

In [77]:
abnormal_play_key = foo(s1)+foo(s2)+foo(s3)

In [80]:
abnormal_play_key = list(set(abnormal_play_key))

In [87]:
_map_tracks = map_tracks.withColumn('season', 
                                    f.when(map_tracks.PlayKey.isin(abnormal_play_key) 
                                           & map_tracks.season.isNull(),1))

In [91]:
# all normal orientation record will be treated as session 2
_map_tracks = _map_tracks.withColumn('season', 
                                    f.when(_map_tracks.season.isNull()
                                           & ~_map_tracks.PlayKey.isin(abnormal_play_key),2))

In [93]:
_map_tracks.persist(StorageLevel.MEMORY_ONLY)

DataFrame[PlayKey: string, time: float, event: string, x: string, y: string, dir: string, dis: string, o: string, s: string, snap_time: string, snap_x: string, snap_y: string, last_time: float, last_event: string, GameID: string, season: int]

In [95]:
print(_map_tracks.count())

17304958


In [ ]:
# Change orientation for season 1
# tracks['o'] = np.where(tracks.Season == 1,
#                             np.mod(tracks.o+90, 360),
#                             tracks.o
#                             )


In [108]:
_map_tracks = _map_tracks.withColumn("s", _map_tracks.s.cast('float').alias("s"))
_map_tracks = _map_tracks.withColumn("o", _map_tracks.o.cast('float').alias("o"))
_map_tracks = _map_tracks.withColumn("dir", _map_tracks.dir.cast('float').alias("dir"))

In [109]:
_map_tracks.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: float (nullable = true)
 |-- event: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- dir: float (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: float (nullable = true)
 |-- s: float (nullable = true)
 |-- snap_time: string (nullable = true)
 |-- snap_x: string (nullable = true)
 |-- snap_y: string (nullable = true)
 |-- last_time: float (nullable = true)
 |-- last_event: string (nullable = true)
 |-- GameID: string (nullable = true)
 |-- season: integer (nullable = true)



In [113]:
adjusted_tracks = _map_tracks.withColumn('o',f.when(f.col('season') == 1, (f.col('o')+90)%360))

In [114]:
adjusted_tracks.show()

+-----------+----+---------+-----+-----+------+----+----+----+---------+------+------+---------+--------------------+--------+------+
|    PlayKey|time|    event|    x|    y|   dir| dis|   o|   s|snap_time|snap_x|snap_y|last_time|          last_event|  GameID|season|
+-----------+----+---------+-----+-----+------+----+----+----+---------+------+------+---------+--------------------+--------+------+
|26624-13-32|19.4|     null|19.85| 23.9|267.19| 0.0|null|0.29|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|26624-13|     2|
|26624-13-32|19.5|ball_snap|19.85| 23.9|266.62| 0.0|null|0.51|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|26624-13|     2|
|26624-13-32|19.6|     null|19.84| 23.9|266.04| 0.0|null|0.68|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|26624-13|     2|
|26624-13-32|19.7|     null|19.84| 23.9|265.88| 0.0|null| 0.8|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|26624-13|     2|
|26624-13-32|19.8|     null|19.84| 23.9|265.98| 0.0|null|0.89|

In [39]:
temp = last_event_tracks[last_event_tracks.PlayKey,last_event_tracks.time,last_event_tracks.event]

In [41]:
temp.show()

+-----------+----+--------------------+
|    PlayKey|time|               event|
+-----------+----+--------------------+
|  26624-1-6| 5.0|              tackle|
| 26624-1-70|24.7|              tackle|
|26624-11-18|19.3|       out_of_bounds|
| 26624-18-9| 8.8|           ball_snap|
|26624-23-24|18.3|pass_outcome_inco...|
|26624-26-28|24.9|pass_outcome_inco...|
| 26624-6-23| 5.7|              tackle|
|  26624-7-2|33.3|        penalty_flag|
| 26624-7-49| 8.8|           ball_snap|
| 27363-1-53|16.8|              tackle|
| 27363-1-62|18.6|pass_outcome_touc...|
|27363-10-63| 9.9|           ball_snap|
|27363-19-42|22.4|              tackle|
|27363-21-59|21.4|              tackle|
|27363-27-45|19.5|              tackle|
|  27363-3-6|16.8|              tackle|
|27363-30-51|51.2|        penalty_flag|
| 27363-4-62|18.4|              tackle|
| 27363-5-23| 9.1|             handoff|
|  27363-6-3|31.3|              tackle|
+-----------+----+--------------------+
only showing top 20 rows



In [100]:
# speed
# player_mean_speed_df = df.rdd.map(lambda x: [x[0].split('-')[0], float(x[-1])]).toDF(["playerKey","s"])
# # remove extreme small speed
# player_mean_speed_df = player_mean_speed_df.rdd.filter(lambda x:x.s >= 1).toDF(["playerKey","s"])
# # get average speed
# player_mean_speed = player_mean_speed_df.groupBy('playerKey').agg(f.mean("s").alias('mean'))
# result = player_mean_speed.collect()

In [88]:
# convert string to float
df = df.withColumn("s", df.s.cast('float').alias("s"))
df = df.withColumn("o", df.o.cast('float').alias("o"))
df = df.withColumn("dir", df.dir.cast('float').alias("dir"))

In [89]:
df.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- dir: float (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: float (nullable = true)
 |-- s: float (nullable = true)



In [90]:
# add new col.
# compute the difference between o and dir
def add_o_dir_diff(x):
    x = x.asDict()
    x['o_dir_diff'] = min([abs(x['o']-x['dir']),abs(x['o']-(x['dir']-360)),abs(x['o']-(x['dir']+360))])
    
    x['is_forward_move'] = False
    x['is_sideway_move'] = False
    x['is_backward_move'] = False
    
    if x['o_dir_diff'] <= 45:
        x['move_type'] = 'forward'
        x['is_forward_move'] = True
    elif x['o_dir_diff'] <= 135:
        x['move_type'] = 'sideway'
        x['is_sideway_move'] = True
    else:
        x['move_type'] = 'backward'
        x['is_backward_move'] = True
        
    return Row(**x)

rdd = df.rdd.map(add_o_dir_diff)

In [91]:
df = rdd.toDF()

In [99]:
# get the distinct event in player trace data
# res = df.select('event').distinct().collect()

In [101]:
# we only focus on ball-snap (the most common event in player trace data)
# rename some columns
_df = df.rdd.filter(lambda x: x.event == 'ball-snap').toDF()
_df = _df.withColumnRenamed('x','x_snap').withColumnRenamed('y','y_snap').withColumnRenamed('time','time_snap')

KeyboardInterrupt: 

In [ ]:
# Remove any data for a play 0.1 second before snap
df = df.merge(_df, on='PlayKey', how='left')
df = df.rdd.filter(lambda x: x.time >= x.time_snap-0.1)

# Remove any data for a play 0.1 second after snap
# TODO